In [1]:
import sys, os, io, json, numpy as np, random, time
#from sklearn.metrics import roc_auc_score
import xgboost as xgb
import tensorflow as tf
from split import split_graph, split_interface, make_split, make_split_quick, make_gax
import graphviz
from sklearn import metrics

/home/zomb-vision/envs/env35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zomb-vision/envs/env35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zomb-vision/envs/env35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zomb-vision/envs/env35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zomb-vision/envs/env35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/zomb-visi

In [2]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label = (pool[name] for name in ['features', 'f_time', 'label'])

In [3]:
np.random.seed(42)
features = np.random.normal(loc=features_bool*1.0, scale=1.0)
#features = features[:299999,:]; label = label[:299999, :]
features_swap = np.concatenate([features[:,7:8], features[:,1:7], features[:,0:1], features[:,8:]], axis=1)

In [4]:
%%time
make_split(np.zeros(label.shape), features, label)

CPU times: user 445 ms, sys: 581 ms, total: 1.03 s
Wall time: 1.11 s


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124402703897527,
 'best_delta_down': -0.8534899433501377,
 'best_delta_up': -0.7958979221951163,
 'best_feature_index': 11,
 'best_index': 261521,
 'best_loss': 183732.0811169258,
 'current_loss': array([207944.15416798]),
 'thr': 1.3039966661872284}

In [5]:
%%time
make_split(np.zeros(label.shape), features_swap, label)

CPU times: user 139 ms, sys: 6.38 ms, total: 146 ms
Wall time: 230 ms


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124402703897527,
 'best_delta_down': -0.8534899433501377,
 'best_delta_up': -0.7958979221951163,
 'best_feature_index': 11,
 'best_index': 261521,
 'best_loss': 183732.0811169258,
 'current_loss': array([207944.15416798]),
 'thr': 1.3039966661872284}

In [6]:
class EMatrix:
    def __init__(self, features, label, bias=None, gax=None, splitgax = False):
        self.bias = bias
        self.features = features
        self.label = label
        self.gax = gax
        self.splitgax = splitgax

In [7]:
class LeafData:
    def __init__(self, info):
        self.val = info['prediction'] * info['learning_rate']
        self.train_size = info['ematrix'].label.shape[0]
        self.avg_target = np.mean(info['ematrix'].label, axis=0)[0]
        
    def to_text(self, floatformat = '.6f'):
        return ('{:'+ floatformat + '} ({})\n({:'+floatformat+'})').format(self.val, self.train_size, self.avg_target)
    
    def shape(self):
        return 'box'

class SplitData:
    def __init__(self, val):
        self.val = val
        
    def to_text(self, floatformat = '.4f'):
        return ('f_{{{ind}}} < {thr:'+floatformat+'}').format(ind=self.val['best_feature_index'], thr=self.val['thr'])
    
        
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.depth = 0
        self.val = None
        self.id = None
        
    def to_text(self, floatformat = '.6f'):
        return self.val.to_text(floatformat)
    
    def shape(self):
        return 'circle'
        

In [8]:
def init_id_helper(node, current_id):
    node.id = current_id[0]
    current_id[0] += 1
    if not isinstance(node, TreeNode):
        return
    init_id_helper(node.left, current_id)
    init_id_helper(node.right, current_id)

def init_id(root):
    current_id = [0]
    init_id_helper(root, current_id)
    return current_id[0]

In [9]:
def init_arrays_helper(node, arrays):
    if not isinstance(node, TreeNode):
        arrays['is_leaf'][node.id] = 1
        arrays['leaf_data'][node.id, 0] = node.val  # Leaf
        return
    init_arrays_helper(node.left, arrays)
    init_arrays_helper(node.right, arrays)
    arrays['yes_node'][node.id] = node.left.id
    arrays['no_node'][node.id] = node.right.id
    arrays['thresholds'][node.id] = node.val.val['thr']
    arrays['features'][node.id] = node.val.val['best_feature_index']
    arrays['is_leaf'][node.id] = 0
    arrays['depths'][node.id] = node.depth
    
def init_arrays(root, n):
    def empty_array():
        return np.zeros(n, dtype=np.int32)
    arrays = dict(features=empty_array(),
                  thresholds=np.zeros(n, dtype=np.float32),
                  yes_node=empty_array(),
                  no_node=empty_array(),
                  is_leaf=empty_array(),
                  depths=empty_array(),
                  leaf_data=np.zeros((n,1), dtype=np.float32)
                 )
    init_arrays_helper(root, arrays)
    arrays['treedepth'] = np.max(arrays['depths'])
    return arrays

In [10]:
def prior_finish(params, info, parent):
    if parent is None:
        return False
    return params['max_depth'] <= parent.depth

def post_finish(params, info, split_info, parent):
    if split_info['left_info']['ematrix'].label.shape[0] < 2:
        return True
    if split_info['right_info']['ematrix'].label.shape[0] < 2:
        return True
    return False

In [11]:
def new_ax(ax, cond):
    reindex = np.cumsum(cond)-1
    axT = ax.T
    return reindex[axT][cond[axT]].reshape((ax.shape[1], -1)).T
time1 = 0
time2 = 0
time3 = []
def split_ematrix(ematrix, depth):
    global time1, time2, time3
    if ematrix.gax is not None:
        start = time.clock()
        split_info = make_split_quick(ematrix.bias, ematrix.features, ematrix.label, ematrix.gax, 
                                      profile_file = ('profile_cumsum2.json' if depth==1 else None))
        dif = time.clock() - start
        time1 += dif
        time3 += [dif]
    else:
        split_info = make_split(ematrix.bias, ematrix.features, ematrix.label)
    start = time.clock()
    thr = split_info['thr']
    features = ematrix.features
    bias = ematrix.bias
    label = ematrix.label
    best_feature = features[:, split_info['best_feature_index']] 
    cond_left = best_feature < thr
    cond_right = np.logical_not(cond_left)
    if ematrix.gax is not None:
        ax_left = new_ax(ematrix.gax, cond_left)
        ax_right = new_ax(ematrix.gax, cond_right)
    else:
        ax_left = None
        ax_right = None
    left_ematrix = EMatrix(features[cond_left, :], label[cond_left], bias[cond_left], gax=ax_left)
    right_ematrix = EMatrix(features[cond_right, :], label[cond_right], bias[cond_right], gax=ax_right)
    left_info = {'prediction': split_info['best_delta_up'], 'ematrix': left_ematrix}
    right_info = {'prediction': split_info['best_delta_down'], 'ematrix': right_ematrix}
    split_info['left_info'] = left_info
    split_info['right_info'] = right_info
    time2 += time.clock() - start
    return left_info, right_info, split_info

In [12]:
def build_tree_helper(params, info, parent):
    info['learning_rate'] = params['learning_rate']
    if parent and parent.depth < 6:
        print("{d}".format(d=parent.depth) 
              if parent else '---',
              "".format(shape=info['ematrix'].label.shape[0]),
              end=' ', file=sys.stderr)
    if prior_finish(params, info, parent):
        return LeafData(info)
    node = TreeNode()
    node.depth = parent.depth + 1 if parent else 1
    
    left_info, right_info, split_info = split_ematrix(info['ematrix'], node.depth)
    if post_finish(params, info, split_info, parent):
        #print(split_info['right_info']['ematrix'].label.shape[0])
        #print(split_info)
        return LeafData(info)

    node.val = SplitData(split_info)
    node.left = build_tree_helper(params, left_info, node)
    node.right = build_tree_helper(params, right_info, node)  
    return node

In [13]:
def build_tree(params, ematrix):
    info = {'ematrix': ematrix}
    if ematrix.splitgax and ematrix.gax is None:
        ematrix.gax = make_gax(ematrix.features)
    return build_tree_helper(params, info=info, parent=None)

In [14]:
def tree_apply(tree_arrays, features):
    qi = np.zeros(features.shape[0], dtype=np.int32)
    for current_depth in range(tree_arrays['treedepth']):
        fi = tree_arrays['features'][qi]
        f = np.choose(fi, features.T)
        t = tree_arrays['thresholds'][qi]
        #print(qi, fi, f, t)
        #if current_depth == 0: 
        #    print(fi, f.shape, features.shape, f)
        answer = (f < t)*1
        new_qi = answer*tree_arrays['yes_node'][qi] + (1-answer)*tree_arrays['no_node'][qi]
        qi = new_qi
    leaf_data = tree_arrays['leaf_data'][qi, 0]
    return leaf_data

In [15]:
features.shape, label.shape

((300000, 15), (300000, 1))

In [17]:
%%time
start_params = {'max_depth': 4, 'learning_rate':0.3}

time1 = time2 =0
time3 = []
tree = build_tree(start_params, EMatrix(features, label, np.zeros(label.shape), splitgax=True))
print("\nsplit time = {:.5f}, {:.5f}, [{}]".format(time1, time2, ", ".join("{:.5f}".format(tt) for tt in time3)), file=sys.stderr)
tree_arrays = init_arrays(tree, init_id(tree))

1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  

CPU times: user 2.87 s, sys: 231 ms, total: 3.1 s
Wall time: 2.54 s


3  4  4  
split time = 2.40035, 0.62980, [0.33057, 0.29493, 0.22638, 0.18933, 0.13573, 0.18924, 0.17625, 0.10090, 0.13211, 0.12243, 0.11987, 0.10325, 0.09075, 0.09061, 0.09797]


In [18]:
pred = tree_apply(tree_arrays, features)
auc1 = metrics.roc_auc_score(label[:, 0], pred)
auc0 = metrics.roc_auc_score(label[:, 0], np.zeros(label.shape[0]))
auc1, auc0

(0.5139355603899424, 0.5)

In [19]:
#tree.left.left.left.left.val, tree.val

In [20]:
def tree2gv(tree):
    result = graphviz.Graph('ni')
    #result.attr(size='12,0')
    tree2gv_helper(tree, result, '')
    return result

In [21]:
def tree2gv_helper(node, result, id):
    idn = id
    result.node(idn, node.to_text(), shape='box') # node.shape())
    if isinstance(node, LeafData):
        return
    if node.left is not None:
        idl = id + '0'
        tree2gv_helper(node.left, result, idl)
        result.edge(idn, idl)
    if node.right is not None:
        idr = id + '1'
        tree2gv_helper(node.right, result, idr)
        result.edge(idn, idr)
    
    

In [22]:
#tree2gv(tree)

In [23]:
class EBooster:
    def __init__(self, forest):
        self.forest = forest
    
    def predict(self, features, tree_limit = None):
        pred = np.zeros(features.shape[0], dtype=np.float32)
        for tree, tree_arrays in (self.forest if tree_limit is None else self.forest[:tree_limit]):
            pred = pred + tree_apply(tree_arrays, features)
        return pred 
    
def train(params, ematrix, num_boost_round = 10):
    start_params = {'max_depth': 5, 'learning_rate': 0.3, 'splitgax': False} 
    start_params.update(params)
    
    if start_params['splitgax'] and ematrix.gax is None:
        ematrix.gax = make_gax(ematrix.features)
    
    forest = []
    bias = np.zeros(ematrix.label.shape)
    for r in range(num_boost_round):
        print("\n{} round".format(r), file=sys.stderr)
        tree = build_tree(start_params, EMatrix(ematrix.features, ematrix.label, bias, gax=ematrix.gax, splitgax=start_params['splitgax']))
        #print("tree ok, bias shape = {}".format(bias.shape), file=sys.stderr)
        tree_arrays = init_arrays(tree, init_id(tree))
        bias_delta = tree_apply(tree_arrays, features)
        #print("apply ok, bias delta shape = {}".format(bias_delta.shape), file=sys.stderr)
        bias = bias + np.reshape(bias_delta, newshape=bias.shape)
        forest.append((tree, tree_arrays))
        #print("forest appended", file=sys.stderr)
        
    return EBooster(forest)

In [24]:
%%time
boost_params = {'max_depth': 4, 'learning_rate':0.3, 'splitgax': True}
booster = train(boost_params, EMatrix(features, label), num_boost_round=7)


0 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  3  4  4  
1 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
2 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
3 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
4 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
5 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  
6 round
1  2  3  4  4  3  4  4  2  3  4  4  3  4  4  1  2  3  4  4  3  4  4  2  3  4  4  3  

CPU times: user 19.6 s, sys: 1.49 s, total: 21.1 s
Wall time: 17.4 s


4  4  

In [31]:
#%%time
#gax13 = make_gax(features)
#print(gax13.shape)

In [32]:
pred2 = booster.predict(features)
metrics.roc_auc_score(label[:, 0], pred2)

0.5310147905131931

In [ ]:
pred2b = booster.predict(features, tree_limit=16)
metrics.roc_auc_score(label[:, 0], pred2)

In [ ]:
np.min(pred2b), np.max(pred2b)

In [ ]:
tree2gv(booster.forest[8][0])

In [ ]:
dmat = xgb.DMatrix(features, label)

In [ ]:
%%time
params = {'max_depth': 4, 'objective': 'binary:logistic', 'eta': 0.3, 'tree_method': 'exact', 'lambda': 1}
model = xgb.train(params, dmat, num_boost_round=20)

In [ ]:
pred_fast = model.predict(dmat)
metrics.roc_auc_score(label[:, 0], pred_fast)

In [ ]:
pred_fast = model.predict(dmat, ntree_limit=16)
metrics.roc_auc_score(label[:, 0], pred_fast)

In [ ]:
qq = -np.log(1/pred_fast-1) 
np.min(qq), np.max(qq)

In [ ]:
diff = np.abs(qq - pred2b)
np.mean(diff > 0.1)

In [ ]:
model.dump_model('xgb.dump')

In [ ]:
tree2gv(booster.forest[15][0])

In [ ]:
!cat xgb.dump | tail -n+$(echo 32*11+22 | bc)